# H₂ Molecular Dissociation Curve via VQE

This notebook demonstrates the complete workflow for computing the H₂ dissociation curve using the Variational Quantum Eigensolver (VQE).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import our package
from h2_vqe import (
    compute_h2_integrals,
    build_qubit_hamiltonian,
    create_ansatz,
    run_vqe,
    compute_dissociation_curve,
)
from h2_vqe.hamiltonian import exact_ground_state_energy
from h2_vqe.visualization import create_dissociation_figure

## 1. Molecular Integrals

First, let's compute the molecular integrals for H₂ at equilibrium bond length (0.74 Å).

In [ ]:
# Compute molecular data at equilibrium
mol_data = compute_h2_integrals(0.74)

print(f"Bond length:        {mol_data.bond_length} Å")
print(f"Basis set:          {mol_data.basis}")
print(f"Number of electrons: {mol_data.n_electrons}")
print(f"Number of orbitals:  {mol_data.n_orbitals}")
print(f"Number of qubits:    {mol_data.n_qubits}")
print(f"\nEnergies (Hartrees):")
print(f"  HF Energy:         {mol_data.hf_energy:.6f}")
print(f"  FCI Energy:        {mol_data.fci_energy:.6f}")
print(f"  Correlation:       {mol_data.correlation_energy():.6f}")

## 2. Qubit Hamiltonian

Now let's build the qubit Hamiltonian using Jordan-Wigner transformation.

In [ ]:
# Build qubit Hamiltonian
qubit_ham = build_qubit_hamiltonian(mol_data)

print(f"Number of qubits:     {qubit_ham.n_qubits}")
print(f"Number of Pauli terms: {qubit_ham.n_terms}")

# Verify exact diagonalization matches FCI
exact_energy, _ = exact_ground_state_energy(qubit_ham)
print(f"\nExact energy (diagonalization): {exact_energy:.8f}")
print(f"FCI energy (PySCF):             {mol_data.fci_energy:.8f}")
print(f"Difference:                     {abs(exact_energy - mol_data.fci_energy):.2e}")

## 3. Variational Ansatze

Let's compare the three available ansatz types.

In [ ]:
# Compare ansatze
for ansatz_type in ["uccsd", "hardware_efficient", "noise_aware"]:
    ansatz = create_ansatz(ansatz_type, n_qubits=4)
    info = ansatz.get_info()
    
    print(f"\n{ansatz_type.upper()}:")
    print(f"  Parameters: {info.n_parameters}")
    print(f"  CNOTs:      {info.n_cnots}")
    print(f"  Depth:      {info.depth}")

## 4. Run VQE

Now let's run VQE with each ansatz type.

In [ ]:
print("Running VQE with different ansatze...\n")
print(f"Reference: FCI Energy = {mol_data.fci_energy:.6f} Ha\n")

results = {}
for ansatz_type in ["uccsd", "noise_aware"]:
    result = run_vqe(mol_data, ansatz_type=ansatz_type, maxiter=100)
    results[ansatz_type] = result
    
    print(f"{ansatz_type.upper()}:")
    print(f"  Energy:      {result.energy:.6f} Ha")
    print(f"  Error:       {result.error*1000:.2f} mHa")
    print(f"  Iterations:  {result.n_iterations}")
    print(f"  Converged:   {result.convergence}\n")

## 5. Full Dissociation Curve

Let's compute the full dissociation curve with multiple bond lengths.

In [ ]:
# Compute dissociation curve (this may take a few minutes)
curve_results = compute_dissociation_curve(
    n_points=10,
    start=0.4,
    stop=2.2,
    ansatz_types=["uccsd", "noise_aware"],
    vqe_maxiter=100,
    verbose=True,
)

## 6. Visualization

Create a publication-quality figure.

In [ ]:
# Create 4-panel figure
fig = create_dissociation_figure(
    curve_results,
    title="H₂ Molecular Dissociation Curve via VQE",
)
plt.tight_layout()
plt.show()

## 7. Results Analysis

In [ ]:
print("\nDissociation Curve Summary")
print("=" * 40)
print(f"Equilibrium distance: {curve_results.equilibrium_distance():.3f} Å")
print(f"Equilibrium energy:   {curve_results.equilibrium_energy():.6f} Ha")
print(f"Dissociation energy:  {curve_results.dissociation_energy():.2f} eV")

print("\nVQE Accuracy:")
for ansatz in curve_results.ansatz_types:
    max_err = curve_results.max_vqe_error(ansatz) * 1000
    mean_err = curve_results.mean_vqe_error(ansatz) * 1000
    print(f"  {ansatz}: max={max_err:.1f} mHa, mean={mean_err:.1f} mHa")

## Conclusion

This notebook demonstrated the complete VQE workflow for H₂:

1. **Molecular Integrals**: Computed using PySCF
2. **Jordan-Wigner**: Transformed to 4-qubit Hamiltonian
3. **Ansatze**: Compared UCCSD, hardware-efficient, and noise-aware
4. **VQE**: Optimized variational parameters
5. **Dissociation Curve**: Computed energy at multiple bond lengths

Key findings:
- VQE satisfies the variational principle (E_VQE ≥ E_FCI)
- UCCSD provides good accuracy with few parameters
- Noise-aware ansatz trades accuracy for NISQ compatibility